In [ ]:
import pandas as pd
import os
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

# Define scoring functions for interactivity aspects
def score_topic_management(texts):
    coherent_topic_words = ['furthermore', 'in addition', 'moreover', 'continuing', 'on the topic']
    transitions = sum(any(word in text.lower() for word in coherent_topic_words) for text in texts)
    if transitions >= len(texts) * 0.5:
        return 5
    elif transitions >= len(texts) * 0.3:
        return 4
    elif transitions >= len(texts) * 0.2:
        return 3
    elif transitions >= len(texts) * 0.1:
        return 2
    else:
        return 1

def score_tone_appropriateness(texts):
    positive_tone_words = ['please', 'thank you', 'appreciate', 'great', 'good', 'nice']
    negative_tone_words = ['annoyed', 'angry', 'frustrated', 'bad', 'hate']
    positive_count = sum(any(word in text.lower() for word in positive_tone_words) for text in texts)
    negative_count = sum(any(word in text.lower() for word in negative_tone_words) for text in texts)
    if positive_count > negative_count and negative_count == 0:
        return 5
    elif positive_count >= negative_count:
        return 4
    elif negative_count > 0 and positive_count == 0:
        return 2
    else:
        return 3

def score_conversation_opening(text):
    opening_words = ['welcome', 'good', 'today', 'start', 'introduce', 'let\'s begin','hello']
    if any(word in text.lower() for word in opening_words):
        return 5
    elif len(text.split()) > 10:
        return 4
    elif len(text.split()) > 5:
        return 3
    else:
        return 2

def score_conversation_closing(text):
    closing_words = ['thank', 'goodbye', 'summary', 'wrap up', 'conclude', 'farewell','fine','cool','end']
    if any(word in text.lower() for word in closing_words):
        return 5
    elif len(text.split()) > 8:
        return 4
    elif len(text.split()) > 5:
        return 3
    else:
        return 2

def refined_evaluate_dialogue_quality(text):
    hesitation_words = ['uh', 'um', 'like', 'you know', 'hmm', 'er', 'ah']
    hesitation_count = sum(text.lower().count(word) for word in hesitation_words)
    sentiment = TextBlob(text).sentiment.polarity
    if hesitation_count == 0 and len(text.split()) > 20 and sentiment > 0.3:
        return 5
    elif hesitation_count <= 2 and len(text.split()) > 15 and sentiment > 0.1:
        return 4
    elif hesitation_count <= 4 and sentiment > -0.2:
        return 3
    elif hesitation_count > 4 and len(text.split()) > 10 and sentiment > -0.5:
        return 2
    else:
        return 1

# Load and evaluate all topic files
def evaluate_dialogue_segments(all_topic_files):
    segment_scores = []
    interactivity_scores = []
    speaker_scores = []

    for topic_file in all_topic_files:
        if os.path.exists(topic_file):
            topic_data = pd.read_csv(topic_file)
            texts = topic_data['text'].fillna("").values

            # Step 1: Evaluate dialogue quality score
            topic_data['refined_quality_score'] = topic_data['text'].apply(refined_evaluate_dialogue_quality)
            average_score = topic_data['refined_quality_score'].mean()
            segment_scores.append({
                'file': os.path.basename(topic_file),
                'average_quality_score': round(average_score, 2),
                'num_utterances': len(topic_data)
            })

            # Step 2: Evaluate interactivity aspects
            topic_management_score = score_topic_management(texts)
            tone_appropriateness_score = score_tone_appropriateness(texts)
            conversation_opening_score = score_conversation_opening(texts[0])
            conversation_closing_score = score_conversation_closing(texts[-1])
            interactivity_scores.append({
                'file': os.path.basename(topic_file),
                'topic_management_score': topic_management_score,
                'tone_appropriateness_score': tone_appropriateness_score,
                'conversation_opening_score': conversation_opening_score,
                'conversation_closing_score': conversation_closing_score
            })

            # Step 3: Score each speaker's performance
            speakers = topic_data['speaker'].unique()
            for speaker in speakers:
                speaker_data = topic_data[topic_data['speaker'] == speaker]
                topic_management_score = score_topic_management(speaker_data['text'].values)
                tone_appropriateness_score = score_tone_appropriateness(speaker_data['text'].values)
                conversation_opening_score = score_conversation_opening(speaker_data['text'].values[0]) if len(speaker_data) > 0 else 0
                conversation_closing_score = score_conversation_closing(speaker_data['text'].values[-1]) if len(speaker_data) > 0 else 0
                speaker_scores.append({
                    'speaker': speaker,
                    'file': os.path.basename(topic_file),
                    'topic_management_score': topic_management_score,
                    'tone_appropriateness_score': tone_appropriateness_score,
                    'conversation_opening_score': conversation_opening_score,
                    'conversation_closing_score': conversation_closing_score
                })

    # Create DataFrames for all steps
    segment_scores_df = pd.DataFrame(segment_scores)
    interactivity_scores_df = pd.DataFrame(interactivity_scores)
    speaker_scores_df = pd.DataFrame(speaker_scores)

    return segment_scores_df, interactivity_scores_df, speaker_scores_df

# Function to save the evaluation results to CSV
def save_results(segment_scores_df, interactivity_scores_df, speaker_scores_df):
    segment_scores_filename = 'your file name'
    interactivity_scores_filename = 'your file name'
    speaker_scores_filename = 'your file name'
    
    segment_scores_df.to_csv(segment_scores_filename, index=False)
    interactivity_scores_df.to_csv(interactivity_scores_filename, index=False)
    speaker_scores_df.to_csv(speaker_scores_filename, index=False)
    
    return segment_scores_filename, interactivity_scores_filename, speaker_scores_filename

# Example usage
all_topic_files = []  # Populate this with paths to the segmented dialogue CSV files
# Evaluate and get the results
dialogue_quality_scores, interactivity_scores, speaker_scores = evaluate_dialogue_segments(all_topic_files)
# Save the results
segment_scores_path, interactivity_scores_path, speaker_scores_path = save_results(dialogue_quality_scores, interactivity_scores, speaker_scores)

# Paths to the results
(segment_scores_path, interactivity_scores_path, speaker_scores_path)
